In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [3]:
chrome_path = 'C:/Users/Redata/Downloads/read/chromedriver_win32/chromedriver.exe' # my directory for chrome driver
driver = webdriver.Chrome(chrome_path)

In [5]:
s = pd.DataFrame(columns=['text', "title", "publish", "link"])

for a in range(0,1000, 10): # we only got maximal 1000 result
    driver.get(f"https://scholar.google.com/scholar?start={a}&q=neet+young+people+neet+OR+young&hl=id&as_sdt=0,5")
    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{random.random()})")
    driver.implicitly_wait(2.2)
    html = driver.page_source
    satu = bs(html)
    i = satu.find_all("div", {"class":"gs_ri"})
    for x in i:
        text = x.get_text() #all text
        #cite = x.get_text().split("Cited by ")[1]
        title = x.find("h3","gs_rt").get_text()
        publish = x.find("div","gs_a").get_text()
        link = str(x.find("h3", {"class":"gs_rt"}))
        s = s.append({'text':text,"title":title, "publish":publish, "link":link}, ignore_index=True)
    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{random.random()})")
    driver.implicitly_wait(2.2)

In [6]:
s.to_csv("google_scholar_datascrape(dirty).csv")

# Little bit cleaning

In [7]:
s["cite"] = s["text"].str.split("Dirujuk ", expand=True)[1]
s["link"] = s["link"].str.split("href=", expand=True)[1].str.split("\"", expand=True)[1].str.replace("amp;", "")
d = s["publish"].str.split("-", expand=True).rename(columns={0:"author", 1:"journal/year", 2:"publisher"})
s[d.columns] = d
s["version"] = s["cite"].str.split("terkait ", expand=True)[1]
s["version"] = s["version"].str.extract(r'(\d+)').fillna(1)
s["citation"] = s["cite"].str.extract(r'(\d+)').fillna(0)
s.head()

,text,title,publish,link,cite,author,journal/year,publisher,3,4,version,citation
0,Not so NEET? A critique of the use of 'NEET'in...,Not so NEET? A critique of the use of 'NEET'in...,"S Yates, M Payne - Journal of youth studies, 2...",https://www.tandfonline.com/doi/abs/10.1080/13...,287 kali Artikel terkait 4 versi,"S Yates, M Payne","Journal of youth studies, 2006",Taylor & Francis,None,None,4,287
1,Social exclusion and the transition from schoo...,Social exclusion and the transition from schoo...,"J Bynner, S Parsons - Journal of vocational be...",https://www.sciencedirect.com/science/article/...,573 kali Artikel terkait 6 versi,"J Bynner, S Parsons","Journal of vocational behavior, 2002",Elsevier,None,None,6,573
2,[BUKU][B] NEET young people and training for w...,[BUKU][B] NEET young people and training for w...,"R Simmons, R Thompson - 2011 - eprints.hud.ac.uk",http://eprints.hud.ac.uk/11546,79 kali Artikel terkait 4 versi Cache,"R Simmons, R Thompson",2011,eprints.hud.ac.uk,None,None,4,79
3,[BUKU][B] NEETs: Young people not in employmen...,[BUKU][B] NEETs: Young people not in employmen...,"M Mascherini, L Salvatore, A Meierkord, JM Jun...",https://www.voced.edu.au/content/ngv:54087,119 kali Artikel terkait 2 versi Cache,"M Mascherini, L Salvatore, A Meierkord, JM Jun...",2012,voced.edu.au,None,None,2,119
4,[HTML][HTML] A cross-sectional exploration of ...,[HTML][HTML] A cross-sectional exploration of ...,"B O'Dea, N Glozier, R Purcell, PD McGorry, J S...",https://bmjopen.bmj.com/content/4/12/e006378?u...,55 kali Artikel terkait 16 versi,"B O'Dea, N Glozier, R Purcell, PD McGorry, J S...","BMJ open, 2014",bmjopen.bmj.com,None,None,16,55


In [27]:
o = s["journal/year"].str.extract(r'(\d+)')
p = s["publisher"].str.extract(r'(\d+)')
q = s[3].str.extract(r'(\d+)')
pd.concat([o, p, q], axis=1)
ui = pd.concat([o, p, q], axis=1).fillna(0)
for i in ui.columns:
    ui[i] = ui[i].astype("int")

zz = ui[((ui.iloc[:, 0] >= 1990) & (ui.iloc[:, 0] <= 2020)) | ((ui.iloc[:, 1] >= 1990) & (ui.iloc[:, 1] <= 2020))
        | ((ui.iloc[:, 2] >= 1990) & (ui.iloc[:, 2] <= 2020))]
s["year_publish"] = zz.sum(axis=1)

#still dirty
s["journal"] = s["journal/year"].str.split(",", expand=True)[[0, 1]].dropna()[0]

In [ ]:
keyword = ["young", "youth", "NEET", "NEETs", "education", "training", "employment", "exclusion", 
           "16", "labour", "labor", "Not in"]

In [32]:
s.to_csv("google_scholar_datascrape.csv")

In [31]:
s.sort_values("year_publish").head(10)

,text,title,publish,link,cite,author,journal/year,publisher,3,4,version,citation,journal,year_publish
399,Status zero youth and the underclassH Williams...,Status zero youth and the underclass,"H Williamson - Youth, the 'underclass' and soc...",https://www.taylorfrancis.com/books/e/97802031...,180 kali Artikel terkait 5 versi Versi HTML,H Williamson,"Youth, the 'underclass' and social exclusion ...",taylorfrancis.com,None,None,5,180,Youth,1997.0
372,"Not in education, employment, or training (NEE...","Not in education, employment, or training (NEE...","J Lee, K Woo - The Korean Labour Market after ...",https://www.taylorfrancis.com/books/e/97802031...,2 kali Artikel terkait Versi HTML,"J Lee, K Woo","The Korean Labour Market after the 1997 …, 2013",taylorfrancis.com,None,None,1,2,The Korean Labour Market after the 1997 …,1997.0
80,"[BUKU][B] Young people not in education, emplo...","[BUKU][B] Young people not in education, emplo...",J Payne - 2000 - voced.edu.au,https://www.voced.edu.au/content/ngv:11552,16 kali Artikel terkait Cache,J Payne,2000,voced.edu.au,None,None,1,16,NaN,2000.0
613,"[PERNYATAAN][C] Young people not in education,...","[PERNYATAAN][C] Young people not in education,...","J Payne - LABOUR MARKET TRENDS, 2000 - … HOUSE...",None,10 kali Artikel terkait,J Payne,"LABOUR MARKET TRENDS, 2000",… HOUSE COMMUNICATIONS PLC,None,None,1,10,LABOUR MARKET TRENDS,2000.0
453,[PERNYATAAN][C] A background to youth disaffec...,[PERNYATAAN][C] A background to youth disaffec...,R Steer - 2000 - … Development Foundation London,None,42 kali Artikel terkait,R Steer,2000,… Development Foundation London,None,None,1,42,NaN,2000.0
534,[BUKU][B] Keeping track mapping and tracking v...,[BUKU][B] Keeping track mapping and tracking v...,"A Canny, AE Green, M Maguire - 2001 - micuat.e...",https://micuat.enovation.ie/handle/10395/2179,31 kali Artikel terkait 8 versi Versi HTML,"A Canny, AE Green, M Maguire",2001,micuat.enovation.ie,None,None,8,31,NaN,2001.0
857,Career guidance and social exclusion: a cautio...,Career guidance and social exclusion: a cautio...,AG Watts - British Journal of Guidance & Couns...,https://www.tandfonline.com/doi/abs/10.1080/03...,132 kali Artikel terkait 4 versi,AG Watts,"British Journal of Guidance & Counselling, 2001",Taylor & Francis,None,None,4,132,British Journal of Guidance & Counselling,2001.0
912,"Disconnected youth?R MacDonald, J Marsh - Jour...",Disconnected youth?,"R MacDonald, J Marsh - Journal of youth studie...",https://www.tandfonline.com/doi/abs/10.1080/13...,186 kali Artikel terkait 4 versi,"R MacDonald, J Marsh","Journal of youth studies, 2001",Taylor & Francis,None,None,4,186,Journal of youth studies,2001.0
1,Social exclusion and the transition from schoo...,Social exclusion and the transition from schoo...,"J Bynner, S Parsons - Journal of vocational be...",https://www.sciencedirect.com/science/article/...,573 kali Artikel terkait 6 versi,"J Bynner, S Parsons","Journal of vocational behavior, 2002",Elsevier,None,None,6,573,Journal of vocational behavior,2002.0
75,[BUKU][B] Missing ConneXions: The Career Dynam...,[BUKU][B] Missing ConneXions: The Career Dynam...,"L Britton, B Chatrik, B Coles, G Craig, C Hylt...",https://eric.ed.gov/?id=ED474777,38 kali Artikel terkait Cache,"L Britton, B Chatrik, B Coles, G Craig, C Hylt...",2002,ERIC,None,None,1,38,NaN,2002.0
